In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely
import shapefile
import flopy
from flopy.utils.gridgen import Gridgen
from flopy.discretization import StructuredGrid, VertexGrid
from flopy.utils.triangle import Triangle as Triangle
from flopy.utils.voronoi import VoronoiGrid
from flopy.utils.gridintersect import GridIntersect

In [ ]:
model_ws = "../temp/structured/"

Load a few raster files

In [ ]:
bottom = flopy.utils.Raster.load("../data_project/aquifer_bottom.asc")
top = flopy.utils.Raster.load("../data_project/aquifer_top.asc")
kaq = flopy.utils.Raster.load("../data_project/aquifer_k.asc")

Load a few shapefiles with geopandas

In [ ]:
river = gpd.read_file("../data_project/Flowline_river.shp")
inactive = gpd.read_file("../data_project/inactive_area.shp")
active = gpd.read_file("../data_project/active_area.shp")
wells = gpd.read_file("../data_project/pumping_well_locations.shp")

Plot the shapefiles with geopandas

In [ ]:
ax = river.plot(color="cyan")
active.plot(ax=ax, color="blue")
inactive.plot(ax=ax, color="white")
wells.plot(ax=ax, color="red", markersize=8);

#### Make a structured model grid

In [ ]:
nlay, nrow, ncol = 3, 160, 80
shape2d, shape3d = (nrow, ncol), (nlay, nrow, ncol)
xlen, ylen = 5000.0, 10000.0
delr = np.full(ncol, xlen / ncol, dtype=float)
delc = np.full(nrow, ylen / nrow, dtype=float)
ttop = np.full(shape2d, 1.0, dtype=float)
tbotm = np.full(shape3d, 0.0, dtype=float)

In [ ]:
base_grid = StructuredGrid(
    delc=delc, delr=delr, nlay=1, nrow=nrow, ncol=ncol, top=ttop, botm=tbotm
)

### Intersect the modelgrid with the shapefiles

#### Create an intersection object

In [ ]:
ix = GridIntersect(base_grid, method="vertex", rtree=True)

#### Intersect inactive and active shapefiles with the modelgrid

After all of the intersection operations, take a look at the data contained in the returned objects

In [ ]:
bedrock = ix.intersect(inactive.geometry[0])
active_cells = ix.intersect(active.geometry[0])

In [ ]:
active_cells[:4], active_cells.dtype

#### Intersect well shapefile with the modelgrid

In [ ]:
well_cells = []
for g in wells.geometry:
    v = ix.intersect(g)
    well_cells += v["cellids"].tolist()

In [ ]:
well_cells

#### Intersect river shapefile with the modelgrid

In [ ]:
river_cells = ix.intersect(river.geometry[0])

In [ ]:
river_cells[:4], river_cells.dtype

### Intersect constant head line with the modelgrid

Use a line with two points to defined the location of the constant head cells. The line verticase are `[(1250, 0.1), (4250, 0.1)]`.

In [ ]:
constant_cells = ix.intersect(
    [(1250, 0.1), (4250, 0.1)], shapetype="linestring"
)

In [ ]:
constant_cells[:4], constant_cells.dtype

### Resample the raster data to the modelgrid

Use the `resample_to_grid()` method on each raster object.

In [ ]:
rtop = top.resample_to_grid(
    base_grid,
    band=top.bands[0],
    method="linear",
    extrapolate_edges=True,
)
rbot = bottom.resample_to_grid(
    base_grid,
    band=bottom.bands[0],
    method="linear",
    extrapolate_edges=True,
)
rkaq = (
    kaq.resample_to_grid(
        base_grid, band=kaq.bands[0], method="linear", extrapolate_edges=True
    )
    * 86400.0
)

### Plot the resampled data 

Plot the aquifer top, bottom, and hydraulic conductivity. Also plot the aquifer thickness.

In [ ]:
mm = flopy.plot.PlotMapView(modelgrid=base_grid)
mm.plot_array(rtop)
mm.plot_grid(lw=0.5, color="0.5");

In [ ]:
mm = flopy.plot.PlotMapView(modelgrid=base_grid)
mm.plot_array(rbot)
mm.plot_grid(lw=0.5, color="0.5");

In [ ]:
mm = flopy.plot.PlotMapView(modelgrid=base_grid)
cb = mm.plot_array(rtop - rbot)
mm.plot_grid(lw=0.5, color="0.5")
plt.colorbar(cb);

In [ ]:
mm = flopy.plot.PlotMapView(modelgrid=base_grid)
cb = mm.plot_array(rkaq)
mm.plot_grid(lw=0.5, color="0.5")
plt.colorbar(cb);

#### Build the model data

_Create the bottom of each model layer_

Assume that the thickness of each layer at a row, column location is equal.

In [ ]:
botm = np.zeros(shape3d, dtype=float)
botm[-1, :, :] = rbot[:, :]
layer_thickness = (rtop - rbot) / nlay
for k in reversed(range(nlay - 1)):
    botm[k] = botm[k + 1] + layer_thickness

_Create the idomain array_

Use the intersection data from the active and inactive shapefiles to create the idomain array

In [ ]:
idomain = np.zeros((nlay, nrow, ncol), dtype=float)
for i, j in active_cells["cellids"]:
    idomain[:, i, j] = 1
for i, j in bedrock["cellids"]:
    idomain[:, i, j] = 0

_Build the well package stress period data_

* The pumping rates are in the `wells` geopandas dataframe
* Pumping rates are in m/sec
* The wells are located in model layer 3

In [ ]:
wells

In [ ]:
well_spd = []
for (i, j), q in zip(well_cells, wells["Q"]):
    well_spd.append([2, i, j, q * 86400.0])
well_spd

_Build the river package stress period data_

* Calculate the length of the river using the `"lengths"` key. 
* The vertical hydraulic conductivity of the river bed sediments is 3.5 m/d.
* The thickness of river bottom sediments at the upstream (North) and downstream (South) end of the river is 0.5 and 1.5 meters, respectively. 
* The river bottom at the upstream and downstream end of the river is 16.5 and 14.5 meters, respectively. The river width at the upstream and downstream end of the river is 5.0 and 10.0 meters, respectively. 
* The river stage at the upstream and downstream end of the river is 16.6 and 15.5 meters, respectively.
* Use the boundname `upstream` for river cells where the upstream end of the river cell is less than 5000 m from the North end of the model. Use the boundname `downstream` for all other river cells. 
* Make sure the river cell is in the upper most layer where the river bottom exceeds the bottom of the cell.

Use the upstream and downstream values to interpolate the river sediment thickness, bottom, width, and stage for each river cell.

The river cells will be conected to model layer 1. The river bottom, width, and stage values should be calculated at the center of the river reach.

In [ ]:
river_kv = 3.5
river_thickness_up, river_thickness_down = 0.5, 1.5
river_bot_up, river_bot_down = 16.5, 14.5
river_width_up, river_width_down = 5.0, 10.0
stage_up, stage_down = 16.6, 15.5

In [ ]:
river_length = river_cells["lengths"].sum()
river_length

In [ ]:
river_thickness_slope = (
    river_thickness_down - river_thickness_up
) / river_length

In [ ]:
river_bot_slope = (river_bot_down - river_bot_up) / river_length

In [ ]:
river_width_slope = (river_width_down - river_width_up) / river_length

In [ ]:
stage_slope = (stage_down - stage_up) / river_length

In [ ]:
boundname = "upstream"
total_length = 0.0
river_spd = []
for idx, (cellid, length) in enumerate(
    zip(river_cells["cellids"], river_cells["lengths"])
):
    if total_length >= 5000.0 and boundname == "upstream":
        boundname = "downstream"
    dx = 0.5 * length
    total_length += dx
    river_thickness = river_thickness_up + river_thickness_slope * total_length
    river_bot = river_bot_up + river_bot_slope * total_length
    river_width = river_width_up + river_width_slope * total_length
    river_stage = stage_up + stage_slope * total_length
    conductance = river_kv * length * river_width / river_thickness

    i, j = cellid
    for k in range(nlay):
        if river_bot - river_thickness > botm[k, i, j]:
            riv_layer = k

    river_spd.append(
        [riv_layer, i, j, river_stage, conductance, river_bot, boundname]
    )
    total_length += dx
river_spd[:2], river_spd[-2:]

_Define river observations_

In [ ]:
riv_obs = {
    "riv_obs.csv": [
        ("UPSTREAM", "RIV", "UPSTREAM"),
        ("DOWNSTREAM", "RIV", "DOWNSTREAM"),
    ],
}

_Define the constant head cells_

Assume the constant head cells are located in all three layers and have values equal to the downstream river stage (`stage_down`). Make sure the constant head stage is greater that the bottom of the layer.

In [ ]:
chd_spd = []
for i, j in constant_cells["cellids"]:
    for k in range(nlay):
        if stage_down > botm[k, i, j] and idomain[k, i, j] > 0:
            chd_spd.append((k, i, j, stage_down))
chd_spd[:2], chd_spd[-2:]

_Define recharge rates_

* The recharge rate is 0.16000000E-08 m/sec

In [ ]:
recharge_rate = 0.16000000e-08 * 86400.0
recharge_rate

#### Build the model

Build a steady-state model using the data that you have created. Packages to create:

* Simulation
* TDIS (1 stress period, `TIME_UNITS='days'`)
* IMS (default parameters)

* GWF model (`SAVE_FLOWS=True`)
* DIS (`LENGTH_UNITS='meters'`)
* IC (`STRT=40.`)
* NPF (Unconfined, same K for all layers, `SAVE_SPECIFIC_DISCHARGE=True`, `icelltype=1`)
* RCH (array based)
* RIV (`BOUNDNAMES=True`,add `RIV` observations for defined boundnames)
* CHD
* WEL
* OC (Save `HEAD ALL` and `BUDGET ALL`)


In [ ]:
name = "project"

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=model_ws, exe_name="mf6")
tdis = flopy.mf6.ModflowTdis(sim, time_units="days")
ims = flopy.mf6.ModflowIms(
    sim, outer_maximum=200, inner_maximum=100, linear_acceleration="bicgstab"
)

In [ ]:
gwf = flopy.mf6.ModflowGwf(
    sim,
    modelname=name,
    save_flows=True,
    newtonoptions="NETWON UNDER_RELAXATION",
)

In [ ]:
dis = flopy.mf6.ModflowGwfdis(
    gwf,
    length_units="meters",
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=rtop,
    botm=botm,
    idomain=idomain,
)
ic = flopy.mf6.ModflowGwfic(gwf, strt=40.0)
npf = flopy.mf6.ModflowGwfnpf(
    gwf, save_specific_discharge=True, k=[rkaq, rkaq, rkaq], icelltype=1
)
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=recharge_rate)
riv = flopy.mf6.ModflowGwfriv(
    gwf, boundnames=True, stress_period_data=river_spd
)
riv.obs.initialize(
    filename=f"{name}.riv.obs",
    digits=9,
    print_input=True,
    continuous=riv_obs,
)
wel = flopy.mf6.ModflowGwfwel(gwf, stress_period_data=well_spd)
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd)
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{name}.hds",
    budget_filerecord=f"{name}.cbc",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)

#### Write the model files

In [ ]:
sim.write_simulation()

#### Run the model

In [ ]:
sim.run_simulation()

#### Post-process the results

Use `gwf.output.` method to get the observations.

In [ ]:
myobs = gwf.riv.output.obs().get_data()
myobs.dtype

In [ ]:
myobs["UPSTREAM"], myobs["DOWNSTREAM"]

Use `gwf.output.` method to get the heads and specific discharge. Make a map and cross-sections of the data using `flopy.plot` methods. Plot specific discharge vectors on the map and cross-sections.

In [ ]:
head = gwf.output.head().get_data()

fpth = "/Users/jdhughes/Documents/Training/python-for-hydrology/notebooks/part1_flopy/data_project/aquifer_top.asc"
np.savetxt(fpth, head[0] + 2.)

In [ ]:
gwf.output.budget().get_unique_record_names()

In [ ]:
spdis = gwf.output.budget().get_data(text="DATA-SPDIS")[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(
    spdis, gwf, head=head
)

In [ ]:
fig, ax = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(5, 8),
    constrained_layout=True,
    subplot_kw=dict(aspect="equal"),
)
mm = flopy.plot.PlotMapView(model=gwf, ax=ax, layer=0)
cb = mm.plot_array(head, masked_values=[1e30], vmin=10, vmax=30)
river.plot(color="cyan", ax=mm.ax)
mm.plot_grid(lw=0.5, color="0.5")
mm.plot_vector(qx, qy, normalize=True)
cs = mm.contour_array(
    head,
    masked_values=[1e30],
    colors="red",
    levels=np.arange(10, 28, 1),
    linestyles=":",
    linewidths=1.0,
)
ax.clabel(
    cs,
    inline=True,
    fmt="%1.0f",
    fontsize=10,
    inline_spacing=0.5,
)
# mm.plot_inactive(ibound=idomain)
plt.colorbar(cb, ax=mm.ax, shrink=0.5)

fig.savefig("../figures/freyberg-structured.png", dpi=300);

In [ ]:
fig, ax = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(5, 8),
    constrained_layout=True,
    subplot_kw=dict(aspect="equal"),
)

mm = flopy.plot.PlotMapView(model=gwf, ax=ax, layer=0)
cb = mm.plot_array(gwf.dis.top.array, masked_values=[1e30], vmin=15, vmax=30)
river.plot(color="cyan", ax=mm.ax)
mm.plot_grid(lw=0.5, color="0.5", zorder=11)
mm.plot_inactive(zorder=10)
cs = mm.contour_array(
    gwf.dis.top.array,
    colors="red",
    levels=np.arange(15, 31, 1),
    linestyles=":",
    linewidths=1.0,
)
ax.clabel(
    cs,
    inline=True,
    fmt="%1.0f",
    fontsize=10,
    inline_spacing=0.5,
)
plt.colorbar(cb, ax=mm.ax, shrink=0.5)

fig.savefig("../figures/freyberg-structured-grid.png", dpi=300);